In [1]:
import pandas as pd

TRAIN_CSV_PATH = 'train.csv'

train = pd.read_csv(
    TRAIN_CSV_PATH, index_col=0
)

train.head(3)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
id,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated


In [2]:
cols = ['title1_zh', 
        'title2_zh', 
        'label']
train = train.loc[:, cols]
train.head(3)

,title1_zh,title2_zh,label
id,,,
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated


Word segmentation

In [3]:
import jieba.posseg as pseg

def jieba_tokenizer(text):
    if type(text) != str:
        text = str(text)
        
    words = pseg.cut(text)
    return ' '.join([word for word, flag in words if flag != 'x'])

train['title1_tokenized'] = train['title1_zh'].apply(jieba_tokenizer)
train['title2_tokenized'] = train['title2_zh'].apply(jieba_tokenizer)

train.iloc[:, [0, 3]].head()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.670 seconds.
Prefix dict has been built successfully.


,title1_zh,title1_tokenized
id,,
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,2017 养老保险 又 新增 两项 农村 老人 人人 可 申领 你 领到 了 吗
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
9,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油


利用 Keras 集成工具包，可以完成文字预处理

In [4]:
train.iloc[:, [0, 3]].head()
train.iloc[:, [1, 4]].head()

,title2_zh,title2_tokenized
id,,
0,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,警方 辟谣 鸟巢 大会 每人 领 5 万 仍 有 老人 坚持 进京
3,深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,深圳 GDP 首 超 香港 深圳 统计局 辟谣 只是 差距 在 缩小
1,GDP首超香港？深圳澄清：还差一点点……,GDP 首 超 香港 深圳 澄清 还 差 一点点
2,去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,去年 深圳 GDP 首 超 香港 深圳 统计局 辟谣 还 差 611 亿
9,吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,吃 了 30 年 食用油 才 知道 一片 大蒜 轻松 鉴别 地沟油


In [5]:
from tensorflow import keras

MAX_NUM_WORDS = 10000
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)

corpus_x1 = train['title1_tokenized']
corpus_x2 = train['title2_tokenized']
corpus = pd.concat([corpus_x1, corpus_x2])
corpus.shape

(641104,)

In [7]:
pd.DataFrame(corpus.iloc[:6], columns=['title'])

,title
id,
0,2017 养老保险 又 新增 两项 农村 老人 人人 可 申领 你 领到 了 吗
3,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
1,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
2,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
9,用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
4,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港


In [8]:
tokenizer.fit_on_texts(corpus)

In [9]:
x1_train = tokenizer.texts_to_sequences(corpus_x1)
x2_train = tokenizer.texts_to_sequences(corpus_x2)


In [11]:
x1_train[:1]

[[217, 1268, 32, 1178, 5967, 25, 489, 2877, 116, 5559, 4, 1850, 2, 13]]

In [12]:
for seq in x1_train[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['2017', '养老保险', '又', '新增', '两项', '农村', '老人', '人人', '可', '申领', '你', '领到', '了', '吗']


In [13]:
for seq in x1_train[:10]:
    print(len(seq), seq[:5], '...')

14 [217, 1268, 32, 1178, 5967] ...
19 [4, 10, 47, 678, 2558] ...
19 [4, 10, 47, 678, 2558] ...
19 [4, 10, 47, 678, 2558] ...
9 [31, 320, 3372, 3062, 1] ...
19 [4, 10, 47, 678, 2558] ...
6 [7, 2221, 1, 2072, 7] ...
19 [4, 10, 47, 678, 2558] ...
14 [1281, 1211, 427, 3, 3245] ...
9 [31, 320, 3372, 3062, 1] ...


In [14]:
max_seq_len = max([len(seq) for seq in x1_train])
max_seq_len

61

In [15]:
MAX_SEQUENCE_LENGTH = 20
x1_train = keras.preprocessing.sequence.pad_sequences(x1_train, maxlen=MAX_SEQUENCE_LENGTH)

x2_train = keras.preprocessing.sequence.pad_sequences(x2_train, maxlen=MAX_SEQUENCE_LENGTH)

x1_train[0]

array([   0,    0,    0,    0,    0,    0,  217, 1268,   32, 1178, 5967,
         25,  489, 2877,  116, 5559,    4, 1850,    2,   13])

In [16]:
for seq in x1_train + x2_train:
    assert len(seq) == MAX_SEQUENCE_LENGTH

print("所有新闻标题的长度都被填充/截断为 {} 个词".format(MAX_SEQUENCE_LENGTH))

所有新闻标题的长度都被填充/截断为 20 个词


In [17]:
x1_train[:5]

array([[   0,    0,    0,    0,    0,    0,  217, 1268,   32, 1178, 5967,
          25,  489, 2877,  116, 5559,    4, 1850,    2,   13],
       [   0,    4,   10,   47,  678, 2558,    4,  166,   34,   17,   47,
        5150,   63,   15,  678, 4502, 3211,   23,  284, 1181],
       [   0,    4,   10,   47,  678, 2558,    4,  166,   34,   17,   47,
        5150,   63,   15,  678, 4502, 3211,   23,  284, 1181],
       [   0,    4,   10,   47,  678, 2558,    4,  166,   34,   17,   47,
        5150,   63,   15,  678, 4502, 3211,   23,  284, 1181],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          31,  320, 3372, 3062,    1,   95,   98, 3372, 3062]])

**One-hot Encoding**

In [18]:
train.label[:5]

id
0    unrelated
3    unrelated
1    unrelated
2    unrelated
9       agreed
Name: label, dtype: object

In [22]:
import numpy as np

# 定义每一个分类对应的索引
label_to_index = {
    'unrelated': 0,
    'agreed': 1,
    'disagreed': 2
}

# 将分类转换为索引
y_train = train.label.apply(lambda x: label_to_index[x])

y_train = np.asarray(y_train).astype('float32')
y_train[:5]

array([0., 0., 0., 0., 1.], dtype=float32)

使用 Keras 做 One-hot encoding:

In [23]:
y_train = keras.utils.to_categorical(y_train)

y_train[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

拆分训练集与验证集

In [24]:
from sklearn.model_selection import train_test_split

VALIDATION_RATIO = 0.1

RANDOM_STATE = 9527

x1_train, x1_val, \
x2_train, x2_val, \
y_train, y_val = train_test_split(
    x1_train, x2_train, y_train, test_size=VALIDATION_RATIO, random_state=RANDOM_STATE)


In [25]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {x1_train.shape}")
print(f"x2_train: {x2_train.shape}")
print(f"y_train : {y_train.shape}")

print("-" * 10)
print(f"x1_val:   {x1_val.shape}")
print(f"x2_val:   {x2_val.shape}")
print(f"y_val :   {y_val.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (288496, 20)
x2_train: (288496, 20)
y_train : (288496, 3)
----------
x1_val:   (32056, 20)
x2_val:   (32056, 20)
y_val :   (32056, 3)
----------
Test Set


In [27]:
for i, seq in enumerate(x1_train[:5]):
    print(f"新聞標題 {i + 1}: ")
    print(seq)
    print()

新聞標題 1: 
[   0    0    0    0    0 4178 2972    9   80   87  717   18  474    4
  968    4  823   14 1436  721]

新聞標題 2: 
[   0    0    0    0    0    0    0    0    0  411  308  809 3142   17
   90  434  191 3713    1    2]

新聞標題 3: 
[   0    0    0    0    0    0    0    0    0    0    0 5212    5 8867
  793 1063 3626  642  337 1172]

新聞標題 4: 
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 2832  244  493  393]

新聞標題 5: 
[   0    0    0    0    0    0    0    0  290  143 2523 2380   46 3120
   70  243  178 2238 3945 1082]



In [28]:
x1_train.shape

(288496, 20)

In [29]:
for i, seq in enumerate(x1_train[:5]):
    print(f"新聞標題 {i + 1}: ")
    print([tokenizer.index_word.get(idx, '') for idx in seq])
    print()

新聞標題 1: 
['', '', '', '', '', '营养师', '补充', '这', '4', '种', '营养', '能', '帮', '你', '降血压', '你', '一样', '都', '不吃', '么']

新聞標題 2: 
['', '', '', '', '', '', '', '', '', '刘涛', '现场', '痛哭', '发飙', '要', '离婚', '直接', '把', '旁边', '的', '了']

新聞標題 3: 
['', '', '', '', '', '', '', '', '', '', '', 'nba', '被', '球星', '诞生', '火箭', '骑士', '同', '抢', '交易']

新聞標題 4: 
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '海口', '飞机', '撒药治', '白蛾']

新聞標題 5: 
['', '', '', '', '', '', '', '', '网', '曝', '杜', '海涛', '与', '沈梦辰', '已', '分手', '疑似', '女方', '劈', '腿']



Word Embedding

In [ ]:
from keras import layers

NUM_EMBEDDING_DIM=3

embedding_layer = layers.Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

Keras 实例

In [36]:
# 基本参数设置，有几个分类
NUM_CLASSES = 3

# 在语料库中有多少个词汇
NUM_MAX_WORDS = 10000

# 一个标题最长有多少个词汇
NUM_MAX_SEQUENCE_LENGTH = 20

# 词向量的维度
NUM_EMBEDDING_DIM = 256

# LSTM 输出的维度
NUM_LSTM_UNITS = 128


# 建立孪生 LSTM 架构（Siamese LSTM）
from keras import Input
from keras.models import Model
from keras.layers import Embedding, LSTM, concatenate, Dense

# 分别定义 2 个新闻标题 A & B 为模型输入节点
# 每个新闻标题都是一个长度为 20 的整数序列
top_input = Input(shape=(NUM_MAX_SEQUENCE_LENGTH, ), dtype='int32')
bm_input = Input(shape=(NUM_MAX_SEQUENCE_LENGTH, ), dtype='int32')

# 对两个输入做词嵌入（embedding）层
# 经过对词嵌入层的处理，两个输入序列都变成了长度为 256 的向量序列
embedding_layer = Embedding(NUM_MAX_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(top_input)
bm_embedded = embedding_layer(bm_input)

# LSTM 层
# 两个新闻标题经过此层后变成了长度为 128 的向量序列
shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)

# 连接层
# 连接层将两个新闻标题的向量序列连接在一起，方便跟全连接层相连
merged = concatenate([top_output, bm_output], axis=-1)

# 全连接层搭配 Softmax Activation
# 可以输出 3 个分类的概率分布
dense = Dense(units=NUM_CLASSES, activation='softmax')
predictions = dense(merged)

# 定义模型的输入输出
# 我们的模型就是将数字序列的输入转换为3个分类的概率分布
model = Model(inputs=[top_input, bm_input], outputs=predictions)


画出模型架构

In [39]:
from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='LR')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


决定如何衡量模型的表现

In [40]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

训练模型并挑选最好的结果

In [41]:
# 决定每次要训练的样本数量（每个 batch 的大小）
BATCH_SIZE = 512

# 决定整个训练集要重复几次（训练多少个 epoch）
NUM_EPOCHS = 10

# 开始训练！
history = model.fit([x1_train, x2_train], # 输入是两个新闻标题的向量序列
                    y_train,             # 标签是新闻标题的分类
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=([x1_val, x2_val], y_val),
                    # 每个 epoch 完成后输出验证集准确度
                    shuffle=True)


Epoch 1/10
564/564 [==============================] - 159s 277ms/step - loss: 0.5426 - accuracy: 0.7390 - val_loss: 0.4551 - val_accuracy: 0.7811
Epoch 2/10
564/564 [==============================] - 147s 260ms/step - loss: 0.4290 - accuracy: 0.8005 - val_loss: 0.4349 - val_accuracy: 0.7954
Epoch 3/10
564/564 [==============================] - 150s 266ms/step - loss: 0.3945 - accuracy: 0.8201 - val_loss: 0.4160 - val_accuracy: 0.8115
Epoch 4/10
564/564 [==============================] - 147s 261ms/step - loss: 0.3719 - accuracy: 0.8323 - val_loss: 0.3941 - val_accuracy: 0.8234
Epoch 5/10
564/564 [==============================] - 145s 258ms/step - loss: 0.3540 - accuracy: 0.8428 - val_loss: 0.3888 - val_accuracy: 0.8249
Epoch 6/10
564/564 [==============================] - 144s 256ms/step - loss: 0.3385 - accuracy: 0.8508 - val_loss: 0.3891 - val_accuracy: 0.8299
Epoch 7/10
564/564 [==============================] - 144s 256ms/step - loss: 0.3243 - accuracy: 0.8578 - val_loss: 0.3903 -

预测并提交结果

In [43]:
import pandas as pd

test = pd.read_csv('test.csv', index_col=0)
test.head(3)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en
id,,,,,,
321187,167562,59521,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,egypt 's presidential election failed to win m...,Lyon! Lyon officials have denied that Felipe F...
321190,167564,91315,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,A message from Saddam Hussein after he was cap...,The Top 10 Americans believe that the Lizard M...
321189,167563,167564,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,Will the United States wage war on Iraq withou...,A message from Saddam Hussein after he was cap...


In [44]:
# 以下步骤分别对新闻标题 A & B 做预处理
# 文本分词 、Word Segmentation
test['title1_tokenized'] = test['title1_zh'].apply(jieba_tokenizer)
test['title2_tokenized'] = test['title2_zh'].apply(jieba_tokenizer)


In [45]:
# 将词汇序列转换为数字序列
x1_test = tokenizer.texts_to_sequences(test['title1_tokenized'])
x2_test = tokenizer.texts_to_sequences(test['title2_tokenized'])

# 为数字序列加入 zero padding
x1_test = keras.preprocessing.sequence.pad_sequences(x1_test, maxlen=MAX_SEQUENCE_LENGTH)
x2_test = keras.preprocessing.sequence.pad_sequences(x2_test, maxlen=MAX_SEQUENCE_LENGTH)

# 利用已经训练好的模型做预测
predictions = model.predict([x1_test, x2_test])

2504/2504 [==============================] - 19s 7ms/step


In [46]:
predictions[:5]

array([[9.8395503e-01, 6.8031877e-05, 1.5976986e-02],
       [9.9851459e-01, 1.0246395e-04, 1.3829672e-03],
       [6.9064534e-01, 3.0933058e-01, 2.4132651e-05],
       [9.8969555e-01, 5.6710717e-04, 9.7372653e-03],
       [9.9493545e-01, 9.0952346e-04, 4.1550971e-03]], dtype=float32)

In [47]:
index_to_label = {v: k for k, v in label_to_index.items()}

test['Category'] = [index_to_label[idx] for idx in np.argmax(predictions, axis=1)]

submission = test.loc[:, ['Category']].reset_index()

submission.columns = ['Id', 'Category']

submission.head()

,Id,Category
0,321187,unrelated
1,321190,unrelated
2,321189,unrelated
3,321193,unrelated
4,321191,unrelated
